In [23]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import shapely.geometry
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

### 1. Connecting with Foursquare Api.

In [24]:
load_dotenv()
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [25]:
office_location = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}
print (office_location)

{'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}


### 2. Defining basskeetball stadiums' query

In [39]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
night_clubs = "4bf58dd8d48988d18b941735"

In [40]:
# By taking information from Foursquare's API, we are looking for basketball stadiums around our office in a radius of 1000m.

parameters = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"night_clubs",
                "radius":1000}

In [41]:
resp = requests.get(url= url_query, params = parameters)
data = json.loads(resp.text)

In [42]:
decoding_data = data.get("response")
decoded = decoding_data.get("groups")[0]
night_clubs = decoded.get("items")

In [43]:
map_ = ["venue","name"]
m_latitude = ["venue","location","lat"]
m_longitude = ["venue","location","lng"]

In [44]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [45]:
print(getFromDict(night_clubs[0],map_))

Lavo


In [47]:
night_clubs_nyc = []
for dic in night_clubs:
    list1 = {}
    list1["name"] = getFromDict(dic,map_)
    list1["latitud"] = getFromDict(dic,m_latitude)
    list1["longitud"] = getFromDict(dic,m_longitude)
    night_clubs_nyc.append(list1)

In [50]:
# Conversion on dataframe that are used on maps construction.
df_night_clubs = pd.DataFrame(night_clubs_nyc)
df_night_clubs.head()

,name,latitud,longitud
0,Lavo,40.762919,-73.971338
1,Studio 54,40.763944,-73.983980
2,JOE & THE JUICE,40.765877,-73.981975
3,Tao,40.762659,-73.971441
4,STK Steakhouse Midtown NYC,40.755262,-73.982888


A total amount of **11 night clubs** in a radius of 1000m from the office.

In [51]:
# We convert our original dataframe into a geodataframe and we add a criteria column.

gdf_night_clubs = gpd.GeoDataFrame(df_night_clubs, geometry = gpd.points_from_xy(df_night_clubs.longitud,df_night_clubs.latitud))
gdf_night_clubs = gdf_night_clubs.assign(name = ["Lavo",
                                        "Studio 54",
                                        "JOE & THE JUICE",
                                        "Tao",
                                        "STK Steakhouse Midtown NYC",
                                        "The Smith",
                                        "The View Restaurant & Lounge",
                                        "Chivas 1801 Club- NYC",
                                        "Flowers by Richard",
                                        "The Mean Fiddler",
                                        "Public House"],
                    criteria = ["night_clubs"]*11)
gdf_night_clubs.head()

,name,latitud,longitud,geometry,criteria
0,Lavo,40.762919,-73.971338,POINT (-73.97134 40.76292),night_clubs
1,Studio 54,40.763944,-73.983980,POINT (-73.98398 40.76394),night_clubs
2,JOE & THE JUICE,40.765877,-73.981975,POINT (-73.98197 40.76588),night_clubs
3,Tao,40.762659,-73.971441,POINT (-73.97144 40.76266),night_clubs
4,STK Steakhouse Midtown NYC,40.755262,-73.982888,POINT (-73.98289 40.75526),night_clubs


In [14]:
# Convert our dataframe on a dict to be able to use it in MongoDB.

In [52]:
gdf_night_clubs["geometry"] = gdf_night_clubs["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_night_clubs

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Lavo,40.762919,-73.971338,"{'type': 'Point', 'coordinates': (-73.97133777...",night_clubs
1,Studio 54,40.763944,-73.983980,"{'type': 'Point', 'coordinates': (-73.98398019...",night_clubs
2,JOE & THE JUICE,40.765877,-73.981975,"{'type': 'Point', 'coordinates': (-73.98197482...",night_clubs
3,Tao,40.762659,-73.971441,"{'type': 'Point', 'coordinates': (-73.97144107...",night_clubs
4,STK Steakhouse Midtown NYC,40.755262,-73.982888,"{'type': 'Point', 'coordinates': (-73.98288809...",night_clubs
5,The Smith,40.755376,-73.968243,"{'type': 'Point', 'coordinates': (-73.96824326...",night_clubs
6,The View Restaurant & Lounge,40.758690,-73.986285,"{'type': 'Point', 'coordinates': (-73.986285, ...",night_clubs
7,Chivas 1801 Club- NYC,40.759405,-73.969916,"{'type': 'Point', 'coordinates': (-73.969916, ...",night_clubs
8,Flowers by Richard,40.764334,-73.985642,"{'type': 'Point', 'coordinates': (-73.985642, ...",night_clubs
9,The Mean Fiddler,40.760313,-73.987162,"{'type': 'Point', 'coordinates': (-73.98716191...",night_clubs


In [53]:
type(gdf_stadiums.loc[0,"geometry"])

dict

### 3. Export the query to Mongo

In [54]:
client = MongoClient() 
db = client.companies
collection = db.night_clubs 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [63]:
data = gdf_night_clubs.to_dict(orient='records')
data[0]

{'name': 'Lavo',
 'latitud': 40.762919168049976,
 'longitud': -73.97133777386868,
 'geometry': {'type': 'Point',
  'coordinates': (-73.97133777386868, 40.762919168049976)},
 'criteria': 'night_clubs'}

In [56]:
collection.insert_many(data)

### 4. Determine the nearest night club.

In [60]:
db.night_clubs 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'night_clubs')

In [61]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.night_clubs .find(filt,proyect).limit(1)
night_clubs = list(result)

In [62]:
night_clubs

[{'name': "Zaro's Bakery", 'latitud': 40.752875, 'longitud': -73.976946}]